In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time
import re
import itertools
import pandas as pd
import numpy as np

# Begining Data Collection

In [3]:
#xpath converter
def xpath_soup(element):
    """
    Generate xpath of soup element
    :param element: bs4 text or node
    :return: xpath as string
    """
    components = []
    child = element if element.name else element.parent
    for parent in child.parents:
        """
        @type parent: bs4.element.Tag
        """
        previous = itertools.islice(parent.children, 0, parent.contents.index(child))
        xpath_tag = child.name
        xpath_index = sum(1 for i in previous if i.name == xpath_tag) + 1
        components.append(xpath_tag if xpath_index == 1 else '%s[%d]' % (xpath_tag, xpath_index))
        child = parent
    components.reverse()
    return '/%s' % '/'.join(components)

In [12]:
browser = webdriver.Chrome()
browser.get("https://www.digitalrock.de/icc_calendar.php")
time.sleep(2)
soup = BeautifulSoup(browser.page_source, "html.parser")
link  = browser.find_element_by_name('year')
# big dataframe to hold all of the information
'ID','Rank','Last Name','First Name', 'Points'
big_df = pd.DataFrame(  {'ID':[0],
                        'Rank':[0],
                        'Last Name':[0],
                        'First Name':[0],
                        'Points':[0],
                        'Event':[0],
                        'Gender':[0],
                        'Year':[0]})

# this for loop looks through the pages of the different years
for i in range(5,30):

    
    link  = browser.find_element_by_name('year')
    link.click()
    j = 2020 - i
    year = str(j)
    link.send_keys(year)
    link.send_keys(Keys.RETURN)
    soup = BeautifulSoup(browser.page_source,'html.parser')
    soup_link = soup.find(attrs = {'name':'ranking'})

#  stuff added by ethan
    if i > 22:
        k = 2
    else:
        k = 5
    rows = soup.select('a[name="ranking"] table:nth-child(2) tr:nth-child(n+2)')
    rows = rows[0].find_all('a',recursive = True)[:k]
   
    
    for row in rows:
        soup_link = row
        gender,event = soup_link.text.split()
        ids = []
        firstnames = []
        lastnames = []
        ranks = []
        countries = []
        points = []
        link = browser.find_element_by_xpath(xpath_soup(soup_link))
        #move tothe link page
        link.click()
        #actual data collection here
        time.sleep(1)
        data_soup = BeautifulSoup(browser.page_source,'html.parser')
        tags = data_soup.find_all('tr')
        tags = tags[1:]



        #getting the data from each row and appending them to a list
        for rank in range(len(tags)):
            tag = tags[rank]
            ids.append(tag.attrs['id'])
            ranks.append(rank+1)
            lastnames.append(tag.td.next_sibling.a.text)
            firstnames.append(tag.td.next_sibling.next_sibling.a.text)
            points.append(float(tag.td.next_sibling.next_sibling.next_sibling.next_sibling.text))

    #going back to Year page
        browser.back()
        #move to next link

        soup_link = soup_link.next_sibling.next_sibling

        #forming the dataframe
        df = pd.DataFrame(list(zip(ids,ranks,lastnames,firstnames,points)),columns = ['ID','Rank','Last Name','First Name', 'Points' ])
        df['Event'] = event
        df['Gender'] = gender
        df['Year'] = j
        big_df = pd.concat([big_df,df])
        
    

big_df.to_csv('Rankings.csv')
browser.close()

KeyboardInterrupt: 

In [29]:
https://www.digitalrock.de/egroupware/ranking/sitemgr/digitalrock/eliste.html#!cat=ICC_F&cup=08_WC

SyntaxError: invalid syntax (<ipython-input-29-22b4049453b4>, line 1)

In [23]:
# scraping for the world cup events
browser = webdriver.Chrome()
browser.get("https://www.digitalrock.de/icc_calendar.php")
time.sleep(2)
soup = BeautifulSoup(browser.page_source, "html.parser")
link  = browser.find_element_by_name('year')
big_df = pd.DataFrame()
for i in range(1,30):
    df = pd.DataFrame()
    link  = browser.find_element_by_name('year')
    link.click()
    j = 2020 - i
    year = str(j)
    link.send_keys(year)
    link.send_keys(Keys.RETURN)
    soup = BeautifulSoup(browser.page_source,'html.parser')
    soup_link = soup.find(attrs = {'name':'ranking'})
    
    if i > 22:
        k = 2
    else:
        k = 6
    rows = soup.select('a[name="ranking"] table:nth-child(2) tr:nth-child(n+2)')
    rows = rows[0].find_all('a',recursive = True)[:k]
    print(year)
    for page in rows:
        soup_link = page
        gender,event = soup_link.text.split()
        link = browser.find_element_by_xpath(xpath_soup(soup_link))
        #move tothe link page
        link.click()
        #actual data collection here
        time.sleep(3)
#         link = browser.find_element_by_class_name('seeAlso')
        soup = BeautifulSoup(browser.page_source,"html.parser")
        time.sleep(3)
        page_link = soup.find(attrs = {'class':'seeAlso'}).li.a
        link = browser.find_element_by_xpath(xpath_soup(page_link))
        link.click()
        time.sleep(1)
        soup = BeautifulSoup(browser.page_source,"html.parser")
        #getting copetition id's
       
        #navigate to table
        Location = soup.find(attrs = {'id':'table'})
        element = browser.find_element_by_xpath(xpath_soup(Location))
#         element = browser.find_element_by_class_name("DrWidget Resultlist")
        df_list = pd.read_html(browser.page_source)
        df_test = pd.DataFrame(df_list[0])
        regex = re.compile(r'result([0-9]+)')
        ids = list(map(lambda r: regex.search(str(r.attrs['class'])).group(1), soup.find_all(class_=regex)))
        col_names = df_test.columns
        col_names = col_names[5:]
        new_col_names = dict(zip(col_names,ids))
        df_test.rename(columns = new_col_names, inplace = True)
        df_test[year] = int(year)
        df_test['event'] = event
        df_test['gender'] = gender
        df_test = df_test.drop(df_test.index[-1])
        #getting athlete id's
        rows = Location.find('tbody').find_all('tr')
        ids = list(map(lambda r: r.attrs['id'], rows))
        df_test['athlete_id'] = ids
        
        time.sleep(1)
        browser.back()
        time.sleep(3)
        browser.back()
        df_test.to_csv(f'Events_data\ {year}_{gender}_{event}.csv')

        
        
big_df.to_csv('yearly_events.csv')      
browser.close()

2019
2018
2017
2016
2015
2014
2013
2012
2011
2010
2009
2008
2007
2006
2005
2004
2003
2002
2001
2000
1999
1998
1997
1996
1995
1994
1993
1992
1991


In [10]:
print(ids)

['8372', '56609', '5089', '14023', '8323', '11392', '10172', '9058', '10775', '10951', '6489', '12432', '12940', '6550', '61376', '10962', '8317', '12604', '12326', '10618', '11373', '8958', '7465', '61368', '14478', '3554', '10963', '9426', '9997', '12601', '8503', '57874', '10788', '9492', '12528', '10279', '12304', '53226', '12629', '6623', '10359', '10825', '9683', '13545', '8315', '11024', '61144', '12607', '9046', '10173', '57746', '12707', '9514', '9758', '14032', '9537', '10365', '50798', '9996', '13867', '13491', '11183', '12078', '57094', '12701', '10404', '11739']


In [11]:
print(len(ids))

67


""
